In [12]:
#Selenium

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Initialize Chrome Driver
driver = webdriver.Chrome()

# List of years
years = list(range(2020, 2024))

# Base URL
base_url = "https://ohiostatebuckeyes.com/sports/mens-basketball/roster/"
#replace with the uni you are interested in

# Create an empty DataFrame to store all players
all_years_players = pd.DataFrame()

# Loop through each year
for year in years:
    # Construct the URL for the year
#     if year == 2024:
#         url = "https://ohiostatebuckeyes.com/sports/mens-basketball/roster/2023"
#         #replace with the university you are interested in
#     else:
    year1 = int(str(year)[-2:])+1
    url = f"{base_url}{year}-{year1}"
        
    # Navigate to the URL
    driver.get(url)
    
    # Click the button to change the view to a table
    try:
        button = WebDriverWait(driver, 4).until(EC.presence_of_element_located((By.ID, "_viewType_table")))
        button.click()
    except:
        print(f"Could not find button for the year {year}")
        continue

    # Wait for the table to load
    try:
        WebDriverWait(driver, 4).until(EC.presence_of_element_located((By.CSS_SELECTOR, '[class^="s-table-body__row"]')))
    except:
        print(f"Timed out waiting for page to load for the year {year}")
        continue

    # Start scraping
    rows = driver.find_elements(by=By.CSS_SELECTOR, value='[class^="s-table-body__row"]')
    all_players = []

    # Your existing loop to extract player information is here
    for row in rows:
        player_info = {}
        class_attribute = row.get_attribute('class')
    
        if '--index-' in class_attribute:
            index = class_attribute.split('--index-')[1]
        else:
            index = None

        tds = row.find_elements(by=By.TAG_NAME, value='td')
    
        player_info['Index'] = index
        player_info['Name'] = tds[1].text if len(tds) > 1 else None
        player_info['Position'] = tds[2].text if len(tds) > 2 else None
        player_info['Height'] = tds[3].text if len(tds) > 3 else None
        player_info['Weight'] = tds[4].text if len(tds) > 4 else None
        player_info['Academic Year'] = tds[5].text if len(tds) > 5 else None
        player_info['Hometown / High School'] = tds[6].text if len(tds) > 6 else None
        player_info['Previous School'] = tds[7].text if len(tds) > 7 else None
        player_info['Year'] = year  # Adding the year
    
        all_players.append(player_info)    
    # Convert to DataFrame and append to all_years_players DataFrame
    year_df = pd.DataFrame(all_players)
    all_years_players = pd.concat([all_years_players, year_df], ignore_index=True)

    # Sleep for a few seconds to be polite to the server
    time.sleep(3)

# Save all_years_players DataFrame to a single CSV
all_years_players.to_csv(f'Ohio_State_basketball_players.csv', index=False, encoding='utf-8')

# Close the browser
driver.quit()